### Load imports and run the no LID case:

In [1]:
%reset
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import runswmm
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
#mongoServer = None   # use the local server
mongoServer = 'mongodb://server.mcgarity.info:27017/'
database = "swmmwise_Ex2_Wakefield_Anna"
collection = 'y16m01d27_testing'
client = MongoClient(mongoServer)
swmmInpFile = "borg_swmm_initial.inp"  # initial SWMM .inp file
lidZeroFile = "lidlist_zero.yaml"
################################################

db = client[database]   # use or create the database
thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
zeroLidDocId = runswmm(zeroLidList,swmmInpStr,thisRunCollection)  # run SWMM on the no LID model for baseline
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage:  peak = %s, volume = %s" % (zeroLidDocId,zeroLidPeak,zeroLidVolume)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
0.0 min, 12.45 sec
Stored record 56a98789abd22f14426c514e for zero LID usage:  peak = 32.82, volume = 15.972


### Loop ALL subcatchment LID numbers over a range:

In [2]:
from copy import deepcopy
for numberForAllLid in range(1,11):
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    for lid in newRunList:
        lid['Number'] = numberForAllLid  # the same number of LIDs for both LIDs will be in every subcatchment
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newPeak = cursor['peak']
    newVolume = cursor['volume']
    peakReduction = zeroLidPeak - newPeak
    volumeReduction = zeroLidVolume - newVolume
    print "Stored record %s for new parameters: peak = %s, volume = %s" % (newRunDocId,newPeak,newVolume)
    print "peakReduction = %s, volumeReduction = %s" % (peakReduction,volumeReduction)
print "FINISHED ALL RUNS"

0.0 min, 14.82 sec
Stored record 56a9885dabd22f14426c514f for new parameters: peak = 31.27, volume = 13.353
peakReduction = 1.55, volumeReduction = 2.619
0.0 min, 14.61 sec
Stored record 56a9886cabd22f14426c5150 for new parameters: peak = 30.17, volume = 12.828
peakReduction = 2.65, volumeReduction = 3.144
0.0 min, 14.93 sec
Stored record 56a9887babd22f14426c5151 for new parameters: peak = 28.13, volume = 12.54
peakReduction = 4.69, volumeReduction = 3.432
0.0 min, 14.50 sec
Stored record 56a9888aabd22f14426c5152 for new parameters: peak = 27.9, volume = 12.339
peakReduction = 4.92, volumeReduction = 3.633
0.0 min, 14.94 sec
Stored record 56a98899abd22f14426c5153 for new parameters: peak = 27.68, volume = 12.162
peakReduction = 5.14, volumeReduction = 3.81
0.0 min, 15.89 sec
Stored record 56a988a9abd22f14426c5154 for new parameters: peak = 27.46, volume = 11.985
peakReduction = 5.36, volumeReduction = 3.987
0.0 min, 14.87 sec
Stored record 56a988b8abd22f14426c5155 for new parameters: p

### To display all of the runs from the database:

In [3]:
runs = db[collection]
cursor = runs.find()
for run in cursor:
    print run
    print ''

{u'lidDict': None, u'swmmStartTime': u'2016-01-27 22:14:05.082951', u'swmmInputFileStr': u'[TITLE]\nExample 2\r\nSurface Drainage System\r\n[OPTIONS]\nFLOW_UNITS\tCFS \nINFILTRATION\tHORTON \nFLOW_ROUTING\tKINWAVE \nLINK_OFFSETS\tDEPTH \nMIN_SLOPE\t0 \nALLOW_PONDING\tNO \nSKIP_STEADY_STATE\tNO \nSTART_DATE\t01/01/2008 \nSTART_TIME\t00:00:00 \nREPORT_START_DATE\t01/01/2008 \nREPORT_START_TIME\t00:00:00 \nEND_DATE\t01/01/2009 \nEND_TIME\t00:00:00 \nSWEEP_START\t01/01 \nSWEEP_END\t12/31 \nDRY_DAYS\t0 \nREPORT_STEP\t00:01:00 \nWET_STEP\t00:01:00 \nDRY_STEP\t01:00:00 \nROUTING_STEP\t0:00:15 \nINERTIAL_DAMPING\tPARTIAL \nNORMAL_FLOW_LIMITED\tSLOPE \nFORCE_MAIN_EQUATION\tH-W \nVARIABLE_STEP\t0.75 \nLENGTHENING_STEP\t0 \nMIN_SURFAREA\t0 \nMAX_TRIALS\t0 \nHEAD_TOLERANCE\t0 \nSYS_FLOW_TOL\t5 \nLAT_FLOW_TOL\t5 \nMINIMUM_STEP\t0.5 \nTHREADS\t1 \n[EVAPORATION]\nTEMPERATURE\t\nDRY_ONLY\tNO \n[TEMPERATURE]\nFILE\t"668594.dat" \nWINDSPEED\tFILE \nSNOWMELT\t34 0.5 0.6 0.0 50.0 0.0 \nADC\tPERVIOUS 1.0 1

### To Reconstitute the SWMM input file for any of the runs above

In [12]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()